<h1><center>TP1 - Ex.1</center></h1>
<p><center>Fevereiro 20, 2024</center></p>


### Estruturas Criptográficas

PG53886, Ivo Miguel Alves Ribeiro

A95323, Henrique Ribeiro Fernandes

#### Trabalho Pratico 1 Exercicio 1


1. Use a package Cryptography   e  o package ascon (instalar daqui) para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:

    a. Autenticação do criptograma e dos metadados (associated data) usando Ascon em modo de cifra.

    b. As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.
    
    c. Para implementar a comunicação cliente-servidor use o package python `asyncio`

In [1]:
import ascon

def generate_key(password, length_Key):
    password = password.encode('utf-8')
    key = ascon.hash(password, variant="Ascon-Xof", hashlength=length_Key)
    return key

def generate_nonce(password, length_nonce):
    password = password.encode('utf-8')
    nonce = ascon.hash(password, variant="Ascon-Xof", hashlength=length_nonce)
    return nonce

def generate_tag(password, length_tag, message, metadata):
    key = generate_key(password, 16)
    tag = ascon.mac(key, message + metadata, "Ascon-Mac")
    tag = ascon.hash(tag, variant="Ascon-Xof", hashlength=length_tag)
    return tag

<p> 
Assim como pedido no ponto b) do exercicio tanto a chave de cifra, como a tag de antencicação e a nonce são gerados por um gerador pseudo aleatório (PRG) usando o Ascon em modo XOF, o que garante que para uma determinada password fornecida é sempre gerada a mesma combinação de bits. 
Assim tanto o Emitter como o Receiver tem acesso às palavras-passe que geram as chaves de cifra e a tag de autenticação para inicialização dos PRG, conseguem ter acesso à mesma combinação de bits em ambas as partes da comunicação.
Conseguimos perceber pela função generate-tag como é feita a autenticação pedida pelo ponto a), onde usamos a palavra-passe de geração de autenticação para gerar uma chave única que é usada juntamente com a concatenação do plaintext e metadados para gerar uma tag, tag essa que, por fim, é usada como chave para gerar uma combinação única de tamanho fixo usando um PRG com Ascon em modo XOF.
</p>

<b>Generate Key with password check!</b>

In [2]:
password = "password"
hash_length = 16

test_key = generate_key(password, hash_length)

print(f"Message: {password}")
print(f"ASCON Hash Output: {test_key.hex()}")

Message: password
ASCON Hash Output: 65f31b63aece5b3ac3d42367623afcf9


<p>Demonstração simples de como a mesma password gera sempre a mesma combinação de bits.</p>

Termos acordados

In [3]:
key_length = 16
nonce_length = 16
tag_length = 16
variant="Ascon-128"

<p>
Estes valores são valores previamente acordados entre o emitter e o reciever para fins de serialização do pacote enviado
</p>

Receiver

In [4]:
import asyncio
import ascon


def serialize(data):
    offset0 = 0 
    offset1 = 2
    t_meta = int.from_bytes(data[offset0:offset1], "big")
    offset0 = offset1
    offset1 = offset1 + t_meta
    metadados = data[offset0:offset1]
    offset0 = offset1
    offset1 = offset1 + tag_length
    received_tag = data[offset0:offset1]
    offset0 = offset1
    offset1 = offset1 + nonce_length
    nonce = data[offset0:offset1]
    offset0 = offset1
    data = data[offset0:]
    return (metadados, received_tag, nonce, data)


async def handle_client(reader, writer, password_to_key, password_to_tag):
    addr = writer.get_extra_info('peername')
    data = await reader.read(200)
    if data:
        print("(Receiver) Pacote recebido: ", data.hex() , " from ", addr)
        (metadados, received_tag, nonce, data) = serialize(data)
        
        key = generate_key(password_to_key, key_length)
        print(f"(Receiver)(key) {key.hex()}")

        message = ascon.decrypt(key, nonce, metadados, data, variant)

        tag = generate_tag(password_to_tag, tag_length, message, metadados)
        print(f"(Receiver)(tag) {tag.hex()}")
        
        if received_tag == tag:
            print("(Receiver) Mensagem autenticada com sucesso")
            print(f"(Receiver) Mensagem decifrada: {message.decode('utf-8')}")
        else:
            print("(Receiver) Mensagem nao autenticada autenticada com sucesso")
    writer.close()


async def receiver(password_to_key, password_to_tag):
    server = await asyncio.start_server(
        lambda r, w: handle_client(r, w, password_to_key, password_to_tag),
        '127.0.0.1', 12347)
    addr = server.sockets[0].getsockname()
    print(f'(Receiver) Serving on {addr}')

    await asyncio.sleep(3)
    server.close()
    await server.wait_closed()

<p>
Receiver tem a função de servidor, é responsavel por abrir a socket de comunicação e aguardar a receção de um pacote.
Após a receção de um pacote passa a informação recebida por uma função de serialize, onde a informação é repartida e dividida conforme os tamanhos anteriormente acordados.
Assim a cada pacote recebido é recebido um conjunto de bits que corresponde aos metadados, um outro com a tag gerada pelo emitter, um outro com a nonce e ainda um com a mensagem cifrada.
Depois é gerada a chave de cifra uma vez que o reciever tem a mesma palavra-chave que o emitter que gera a mesma combinação de bits. Com essa chave, nonce, mensagem cifrada e metadados recebidos no pacote decifra a mensagem.
Para validar a informação recebida, isto é, garantir a integridade da receção da mensagem, o receiver gera uma tag com a informação recebida combinada com a palavra-chave para gerar uma tag única e igual à do emitter, e compara a mesma com a recebida previamente, correspondendo a mensagens válidas apenas as mensagens em pacotes que esta comparação corresponder. 
</p>

Emmiter

In [5]:
import asyncio
import ascon

async def emmiter(message, password_to_key, password_to_nonce, password_to_tag):
    message = message.encode()
    key = generate_key(password_to_key, key_length)
    print(f"(Emitter )(key) {key.hex()}")
    
    nonce = generate_nonce(password_to_nonce, nonce_length) 
    metadados = b"criado para fins de teste"

    reader, writer = await asyncio.open_connection(
        '127.0.0.1', 12347)

    tag = generate_tag(password_to_tag, tag_length, message, metadados)
    print(f"(Emitter )(tag) {tag.hex()}")

    data = ascon.encrypt(key, nonce, metadados, message,  variant)
    t_meta = len(metadados)
    data = t_meta.to_bytes(2, 'big') + metadados + tag + nonce + data
    
    writer.write(data)
    print("(Emitter ) Pacote enviado : ", data.hex())
    await writer.drain()

    writer.close()
    await writer.wait_closed()

Emitter tem a função de cliente, sendo por isso responsavel por cifrar a mensagem e enviar o pacote de dados com toda a informação necessaria para a sua decifragem.
Inicialmente é gerada uma chave com a palavra-chave para geração de chave de cifra, e posteriormente uma nonce tambem recorrendo a um PRG e com uma palavra-chave igualmente fornecida como input.
Após abrir a conexão, do lado de emitter, é gerada a tag de autenticação recorrendo como acima definida à chave de autenticação, mensagem a cifrar e aos metadados.
De seguida a mensagem é cifrada através da função encrypt do Ascon à qual são passados argumentos como metadados, nonce e chave.
Por fim é enviada o pacote respeitando os parametros previamente acordados de tamanho entre o emitter e o reciever.

In [6]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

mensagem = "Hello, server from emitter!"
password_to_key = "password_to_key"
password_to_nonce = "password_to_nonce"
password_to_tag = "password_to_tag"

async def main():
    task1 = asyncio.create_task(receiver(password_to_key, password_to_tag))
    task2 = asyncio.create_task(emmiter(mensagem, password_to_key, password_to_nonce, password_to_tag))
    await asyncio.gather(task1, task2)
    
    
await main()

(Emitter )(key) 6a1b91254d34f868ad888fa554a167c5
(Receiver) Serving on ('127.0.0.1', 12347)


(Emitter )(tag) 5abdf764e22b456f088dda6bed7181bd
(Emitter ) Pacote enviado :  001963726961646f20706172612066696e732064652074657374655abdf764e22b456f088dda6bed7181bd39c3c486592487a7068be171994aa380b045c55d50ebf8ea9904b9a5b9d3deafcd0f6738530b898fd608e47d778d455fa011049b34fe757c9eb77c
(Receiver) Pacote recebido:  001963726961646f20706172612066696e732064652074657374655abdf764e22b456f088dda6bed7181bd39c3c486592487a7068be171994aa380b045c55d50ebf8ea9904b9a5b9d3deafcd0f6738530b898fd608e47d778d455fa011049b34fe757c9eb77c  from  ('127.0.0.1', 62800)
(Receiver)(key) 6a1b91254d34f868ad888fa554a167c5
(Receiver)(tag) 5abdf764e22b456f088dda6bed7181bd
(Receiver) Mensagem autenticada com sucesso
(Receiver) Mensagem decifrada: Hello, server from emitter!


Como era de espectar, uma vez que não houve nenhum ataque comprometendo a integridade do pacote de dados enviado, a mensagem do emitter chega e é decifrada corretamnete por parte do reciever.
Ressaltamos ainda as prints das combinações de bits correspondetes à chave de cifra gerada tanto do lado do emitter como do reciever, que são equivalentes uma vez que a palavra-chave usada para as gerar é também ela identica, assim como a tag de autenticação em ambos, o que permite que a mensagem seja autenticada e revelada pelo reciever.   